In [226]:
from tqdm.notebook import tqdm as notebook
import numpy as np
import pandas as pd
import time
import cvxopt

from numpy import dot, array

import scipy

from scipy import sparse

import matplotlib.pyplot as plt

from cvxopt import matrix, solvers 




class SVM_MULTICLASS:
    def __init__(self):
        
        self.total_Classes = None
        
        self.x_train_data = None
        
        self.mat1 = None
        
        self.mat2= None
        
        self.alpha= None
        
        
        self.C= None
        
        self.degree = None
        
        self.sigma= None
    
    
    def train(self, x_train, y_train,C ,degree ,sigma ):
        
      
        
        opts = {'maxiters' : 50,'solver':'mosek'}
        

        
        total_Classes = len(np.unique(y_train))
        
        self.total_Classes = total_Classes
        
        self.C = C
        
        self.x_train_data = x_train
        
        x_train_copy = x_train
        
        y_train_copy = y_train
        
        self.degree = degree
        
        self.sigma = sigma 
        
        x_dim = x_train.shape[0]*(total_Classes-1)
        y_dim = total_Classes*((x_train.shape[1])+1)

#         mat1 = np.zeros(((x_train.shape[0]*(total_Classes-1)),total_Classes*((x_train.shape[1])+1)))
        
        mat1 = np.zeros((x_dim, y_dim))
        

#         cnt = 0
#         print('step 1')
#         for i in notebook(range(0,x_train_copy.shape[0])):
#             for p in range(0,total_Classes):
#                 if(y_train_copy[i]!=p):   
#                     for j in range(0,total_Classes):
#                         p1 = j*(x_train.shape[1]+1)
#                         p2 = ((j+1)*(x_train.shape[1]+1)) - 1
#                         if(j==y_train_copy[i]): 
#                             mat1[cnt,p1:p2] = x_train_copy[i,:]
# #                             p3 = ((j+1)*(x_train.shape[1]+1))-1
#                             mat1[cnt,p2] = 1.0
# #                             mat1[cnt,(j*(x_train.shape[1]+1)):((j+1)*(x_train.shape[1]+1))-1] = x_train_copy[i,:]
# #                             mat1[cnt,((j+1)*(x_train.shape[1]+1))-1] = 1.0
#                         elif(j==p):
# #                             mat1[cnt,(j*(x_train.shape[1]+1)):((j+1)*(x_train.shape[1]+1))-1] = -1*x_train_copy[i,:]
# #                             mat1[cnt,((j+1)*(x_train.shape[1]+1))-1] = -1.0
#                             mat1[cnt,p1:p2] = -1*x_train_copy[i,:]
#                             mat1[cnt,p2] = -1.0
#                     cnt += 1
            
            
        cnt = 0
        print('step 1')
        for i in notebook(range(0,x_train_copy.shape[0])):
            for z in range(0,total_Classes):
                if(y_train_copy[i]!=z):   
                    for j in range(0,total_Classes):
                        p1 = j*(x_train.shape[1]+1)
                        p2 = ((j+1)*(x_train.shape[1]+1)) - 1
                        if(j==y_train_copy[i]): 
                            mat1[cnt,p1:p2] = x_train_copy[i,:]
#                             p3 = ((j+1)*(x_train.shape[1]+1))-1
                            mat1[cnt,p2] = 1.0
#                             mat1[cnt,(j*(x_train.shape[1]+1)):((j+1)*(x_train.shape[1]+1))-1] = x_train_copy[i,:]
#                             mat1[cnt,((j+1)*(x_train.shape[1]+1))-1] = 1.0
                        elif(j==z):
#                             mat1[cnt,(j*(x_train.shape[1]+1)):((j+1)*(x_train.shape[1]+1))-1] = -1*x_train_copy[i,:]
#                             mat1[cnt,((j+1)*(x_train.shape[1]+1))-1] = -1.0
                            mat1[cnt,p1:p2] = -1*x_train_copy[i,:]
                            mat1[cnt,p2] = -1.0
                    cnt += 1    
        self.mat1= mat1           

        n_rows = mat1.shape[0]

        M1 = sparse.csr_matrix(mat1)
        
        #Intializing dimensions of the matrices 
        x_dim1 = (x_train.shape[0]*(total_Classes-1))
        y_dim1 = total_Classes
        #Initializing matrix 2
        mat2 = np.zeros((x_dim1,y_dim1))
        
        #calculatingkernelizedProduct
        Z, mat2 = self.calculateKernelizedProduct(x_train,y_train,sigma,degree)
        
        
        self.mat2 = mat2

  
        
        e = matrix(Z)
        
        x_dim2 = -1*np.ones((n_rows, 1))
        
        f = matrix(x_dim2)
        
        x_dim3 = np.vstack((np.eye(n_rows)*-1,np.eye(n_rows)))
        
        G = matrix( x_dim3 )
        
        x_dim4 = np.hstack((np.zeros(n_rows), np.ones(n_rows) * C))
        
        h = matrix(x_dim4)
        
        
#         opts = {'maxiters' : 30,'solver':'mosek'}
        
#         opts = {'maxiters' : 50,'solver':'mosek'}
        
        
        
        solution = solvers.qp(e, f, G, h ,options = opts)
        
        
        alpha1 = np.array(solution['x']) 
        
        
        self.alpha = alpha1
        
        
        
    def predict(self,x_test):
    
        total_Classes = self.total_Classes
        x_train_data = self.x_train_data
        degree = self.degree
        sigma = self.sigma
        alpha1 = self.alpha
        mat1= self.mat1
        mat2= self.mat2

        pred_labels = []
        n = x_test.shape[0]
        print('\n Calculating prediction values')
        for i in notebook(range(0,n)):
            
                label_score = (alpha1.T)@(self.calculateKernelizedProductforPrediction(x_train_data,x_test,mat2,i,total_Classes,sigma,degree))
                maxScoreLabel = np.argmax(label_score)
                pred_labels.append(maxScoreLabel)    
            
        return pred_labels
    
    
    def calculateKernelizedProductforPrediction(self,x_train,x_test,mat2,currentValIndex,total_Classes,sigma,degree):
#         X
        x_train_copy = x_train
        m = (x_train.shape[0]*(total_Classes-1))
        Z = np.zeros((m,total_Classes))    
        matrix2 = (currentValIndex+1)*np.eye(total_Classes)
        t_coeff = (1/(x_train_copy.shape[1]+1))
        
        for i in (range(0,m)):
                for j in range(0,total_Classes):
                    cnt = 0
                    for k in range(0,total_Classes):
                        if(cnt == 2):
                            break
                        else:
                            t1 = int(mat2[i,k])
                            t4 = int(matrix2[k,j])
                            if(t1 > 0):
                                cnt+=1
                                if(t4>0):
                                    c1 = x_train_copy[t1-1,:]
                                    c2 = x_test[t4-1,:]
                                    b = c1 - c2
                                    a = np.linalg.norm(b)**2
                                    Z[i,j] += np.exp(-1*a/(2*sigma**2))   
                            elif(t1<0):
                                cnt+=1
                                if(t4>0):
                                    c1 = np.append(x_test[t4-1,:],1.0)
                                    c2 = np.append(-1*x_train_copy[-t1-1,:],-1.0)
                                    b = c1 - c2
                                    a = np.linalg.norm(b)**2
                                    Z[i,j] += np.exp(-1*a/(2*sigma**2))                    

            
        return Z  
    ##ASSUMES RBF KERNEL
    def calculateKernelizedProduct(self,x_train,y_train,sigma,degree):
        x_train_copy = x_train
        y_train_copy = y_train
        total_Classes = len(np.unique(y_train))
        x_dim = (x_train.shape[0]*(total_Classes-1))
        y_dim = total_Classes 
        mat2 = np.zeros((x_dim,y_dim))
        cnt = 0
#         m = (x_train.shape[0]*(total_Classes-1))
        s = x_train_copy.shape[0]
        non_Zero_Values = []
        print('\n Step 2')
        
        for i in notebook(range(0, s)):
            for z in range(0,total_Classes):
                if(y_train_copy[i]!=z):   
                    t_arr = []
                    for j in range(0,total_Classes):
                        if(j==y_train_copy[i]):
                            t_arr.append(j)
                            t_arr.append(i+1)
                            mat2[cnt,j] = i+1
                        elif(j==z):
                            t_arr.append(j)
                            t_arr.append(-(i+1))
                            mat2[cnt,j] = -(i+1)
                    cnt += 1
                    non_Zero_Values.append(t_arr)
                    
        
        matrix2 = mat2.T
        M2 = scipy.sparse.coo_matrix(mat2)
        M3 = scipy.sparse.coo_matrix(mat2.T)
        m = (x_train.shape[0]*(total_Classes-1))
        Z = np.zeros((m,m))
        t_coeff = (1/( x_train_copy.shape[1]+1))
        
        
        
        matrix2 = matrix2.tolist()
        
        print('\n Step 3')
        for i in notebook(range(0,m)):
                for j in range(0,i+1):
                    
                    
                    x1 = 1
                    x2 = 1
                    x3 = 1
                    x4 = 1
                    
                    s1 = non_Zero_Values[i][1]
                    
                    s2 =non_Zero_Values[i][3]
                    
                    loc_s1 = non_Zero_Values[i][0]
                    
                    loc_s2 = non_Zero_Values[i][2]
                    
                    s3 = matrix2[loc_s1][j]
                    s4 = matrix2[loc_s2][j]
                    
                    e = 2.718281828459045
                    
                    if(s1 < 0):
                        x1 = -1
                    if(s2 < 0):
                        x2 = -1
                    if(s3 < 0):
                        x3 = -1
                    if(s4 < 0):
                        x4 = -1
                        
                    if(s3 != 0):
                        c1 = np.append(x1*x_train_copy[int(x1*s1)-1,:],x1)
                        c2 = np.append(x3*x_train_copy[int(x3*s3)-1,:],x3)
                        b = c1 - c2
                        a = np.linalg.norm(b)**2                      
                        Z[i,j] += (e)**(-1*(a)/(2*sigma**2))
                        
                    if(s4 != 0):
                        c1 = np.append(x2*x_train_copy[int(x2*s2)-1,:],x2)
                        c2 = np.append(x4*x_train_copy[int(x4*s4)-1,:],x4)
                        b = c1 - c2
                        a = np.linalg.norm(b)**2
                        Z[i,j] += (e)** (-1*(a)/(2*sigma**2))    
    
        
        
        print('\n Step 4')
        for i in notebook(range(0,m)):
            for j in range(0,i+1):
                if(j>i):
                    Z[i,j]=Z[j,i]  
                    
        print('\n Model Trained')        

        return Z,mat2

In [227]:
dry_bean_data = pd.read_csv('Dry_Bean_Dataset.csv')

codes = {'SEKER': 0, 'HOROZ':1,'CALI':2, 'BARBUNYA':3,'BOMBAY':4,'DERMASON':5, 'SIRA':6}

dry_bean_data['class_new'] = dry_bean_data['Class'].map(codes)

# dry_bean_data = dry_bean_data.head(5000)

# dry_bean_data = dry_bean_data.sort_values(by = 'Area')

# dry_bean_data = dry_bean_data.sample(2000)



# dry_bean_data 

In [228]:
X = dry_bean_data.copy()
columns_to_drop  = ['Class','Area','EquivDiameter','ShapeFactor3','Perimeter','AspectRation'
                   ,'MinorAxisLength','Compactness','MinorAxisLength']
X.drop(columns_to_drop,axis=1,inplace=True)

In [229]:
X_new = X.iloc[:, 0:9]
Y =  X.iloc[:, 9:]
Y.value_counts()

class_new
5            3546
6            2636
0            2027
1            1928
2            1630
3            1322
4             522
dtype: int64

In [230]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.33, random_state=42)

In [231]:

from imblearn.datasets import make_imbalance
# from imblearn.under_sampling import NearMiss
# from imblearn.pipeline import make_pipeline
# from imblearn.metrics import classification_report_imbalanced


RANDOM_STATE = 42
# X_res, y_res  = make_imbalance(
#     X_new,
#     Y,
#     sampling_strategy={0: , 1:10, 2: 10, 3:10, 4:10,5:10,6:10},
#     random_state=RANDOM_STATE,
# )



X_res, y_res  = make_imbalance(
    X_new,
    Y,
    sampling_strategy={0: 10, 1:10, 2: 10, 3:10, 4:10,5:10,6:10},
    random_state=RANDOM_STATE,
)

In [232]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res,test_size=0.33, random_state=RANDOM_STATE)

In [233]:
from sklearn.preprocessing import StandardScaler
import numpy as np
# Scale the features

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))

X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))

y_n = y_train.to_numpy()

y_n1 = y_test.to_numpy()

In [234]:
degree = 4
sigma = 0.4 

C = 10 
a1 = SVM_MULTICLASS()
start_time = time.time()
a1.train(X_train_scaled,y_n,C,degree,sigma)

import pickle
with open('model_svm', 'wb') as f:
    pickle.dump(a1,f)

print('\n Time taken for training in seconds: ', time.time()-start_time )
predictedLabels = a1.predict(X_test_scaled)


step 1


  0%|          | 0/46 [00:00<?, ?it/s]


 Step 2


  0%|          | 0/46 [00:00<?, ?it/s]


 Step 3


  0%|          | 0/276 [00:00<?, ?it/s]


 Step 4


  0%|          | 0/276 [00:00<?, ?it/s]


 Model Trained
     pcost       dcost       gap    pres   dres
 0:  1.0725e+03 -4.7794e+03  6e+03  5e-17  3e-15
 1:  1.4104e+02 -3.2686e+02  5e+02  2e-16  3e-15
 2: -5.0380e+00 -6.7142e+01  6e+01  2e-16  9e-16
 3: -1.8230e+01 -2.4756e+01  7e+00  2e-16  5e-16
 4: -1.8628e+01 -1.8849e+01  2e-01  2e-16  3e-16
 5: -1.8629e+01 -1.8631e+01  2e-03  2e-16  2e-16
 6: -1.8629e+01 -1.8629e+01  2e-05  2e-16  2e-16
 7: -1.8629e+01 -1.8629e+01  2e-07  2e-16  2e-16
Optimal solution found.

 Time taken for training in seconds:  3.1793110370635986

 Calculating prediction values


  0%|          | 0/24 [00:00<?, ?it/s]

In [235]:
predictedLabels = a1.predict(X_test_scaled)


 Calculating prediction values


  0%|          | 0/24 [00:00<?, ?it/s]

In [236]:

from sklearn.metrics import f1_score, accuracy_score
f1_score(y_n1,predictedLabels, average = None)


array([1.        , 0.88888889, 0.8       , 0.75      , 1.        ,
       1.        , 0.5       ])

In [237]:
accuracy_score(y_n1,predictedLabels)


0.875

In [238]:
conMatrix = pd.crosstab(np.array(predictedLabels).ravel(),y_n1.reshape(-1,1).ravel())
conMatrix

col_0,0,1,2,3,4,5,6
row_0,,,,,,,
0,4,0,0,0,0,0,0
1,0,4,0,0,0,0,1
2,0,0,2,1,0,0,0
3,0,0,0,3,0,0,0
4,0,0,0,0,4,0,0
5,0,0,0,0,0,3,0
6,0,0,0,1,0,0,1
